In [63]:
print("ok")

ok


In [64]:
%pwd

'd:\\'

In [65]:
import os

# os.chdir("../")
%pwd

'd:\\'

In [61]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [62]:
def load_pdf_files(data):

    loader = DirectoryLoader(  #Created to load all PDF files from a directory
        data,
        glob = "*.pdf",
        loader_cls=PyPDFLoader,
    )

    documents = loader.load()
    return documents

extracted_data = load_pdf_files("data")


FileNotFoundError: Directory not found: 'data'

In [ ]:
extracted_data[33:34]  #Displaying first two loaded documents

[Document(metadata={'producer': 'calibre 3.42.0 [https://calibre-ebook.com]', 'creator': 'calibre 3.42.0 [https://calibre-ebook.com]', 'creationdate': '2019-07-12T08:02:34+00:00', 'author': 'Sudha Murty', 'title': 'Three Thousand Stitches - PDFDrive.com', 'source': 'data\\Three Thousand Stitches by Sudha Murthy.pdf', 'total_pages': 128, 'page': 33, 'page_label': '34'}, page_content='others\tsay\tit\tcame\tfrom\tMexico.\tAccording\tto\ta\ttheory,\tblack\tpepper\twas\tthe\ningredient\ttraditionally\tused\tin\tIndia\tto\tmake\tour\tfood\thot\tand\tspicy.\tSome\nscholars\tbelieve\tthat\tthe\tsole\tgoal\tof\tthe\tEast\tIndia\tCompany\twas\tto\tacquire\ta\nmonopoly\tover\tIndia’s\tpepper\ttrade,\n\twhich\tlater\tended\tin\tIndia’s\tcolonization.\nBut\twhen\twe\tbegan\tusing\tchillies,\twe\tfound\tthat\tit\ttasted\tbetter\tthan\tblack\npepper.\tTo\tgive\tyou\tan\texample,\twe\trefer\tto\tblack\tpepper\tas\t\nkalu\tmenasu\n\tin\nKannada.\tWe\tgave\ta\tsimilar\tname\tto\tthe\tchilli\tand\tcalle

In [ ]:
print(type(extracted_data))


<class 'list'>


In [ ]:
# [i for i in extracted_data[34].metadata.items()]  #Displaying metadata of first document

[i for i in extracted_data[3]]  #Displaying metadata of first document

[('id', None),
 ('metadata',
  {'producer': 'calibre 3.42.0 [https://calibre-ebook.com]',
   'creator': 'calibre 3.42.0 [https://calibre-ebook.com]',
   'creationdate': '2019-07-12T08:02:34+00:00',
   'author': 'Sudha Murty',
   'title': 'Three Thousand Stitches - PDFDrive.com',
   'source': 'data\\Three Thousand Stitches by Sudha Murthy.pdf',
   'total_pages': 128,
   'page': 3,
   'page_label': '4'}),
 ('page_content',
  'Contents\nPreface\n1.\tThree\tThousand\tStitches\n2.\tHow\tto\tBeat\tthe\tBoys\n3.\tFood\tfor\tThought\n4.\tThree\tHandfuls\tof\tWater\n5.\tCattle\tClass\n6.\tA\tLife\tUnwritten\n7.\tNo\tPlace\tLike\tHome\n8.\tA\tPowerful\tAmbassador\n9.\tRasleela\tand\tthe\tSwimming\tPool\n10.\tA\tDay\tin\tInfosys\tFoundation\n11.\tI\tCan’t,\tWe\tCan\nFollow\tPenguin\nCopyright'),
 ('type', 'Document')]

In [ ]:

# data = [
#     ('id', None),
#     ('metadata',
#      {'producer': 'calibre 3.42.0 [https://calibre-ebook.com]',
#       'creator': 'calibre 3.42.0 [https://calibre-ebook.com]',
#       'creationdate': '2019-07-12T08:02:34+00:00',
#       'author': 'Sudha Murty',
#       'title': 'Three Thousand Stitches - PDFDrive.com',
#       'source': 'data\\Three Thousand Stitches by Sudha Murthy.pdf',
#       'total_pages': 128,
#       'page': 3,
#       'page_label': '4'}),
#     ('page_content', '...'),
#     ('type', 'Document')
# ]

# # Extract producer
# producer = next((value['producer'] for key, value in data if key == 'metadata'), None)
# print(producer)

In [ ]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [ ]:
minimal_docs = filter_to_minimal_docs(extracted_data)
minimal_docs

[Document(metadata={'source': 'data\\Three Thousand Stitches by Sudha Murthy.pdf'}, page_content=''),
 Document(metadata={'source': 'data\\Three Thousand Stitches by Sudha Murthy.pdf'}, page_content=''),
 Document(metadata={'source': 'data\\Three Thousand Stitches by Sudha Murthy.pdf'}, page_content='SUDHA\tMURTY\n\t\nTHREE\tTHOUSAND\tSTITCHES\nOrdinary\tPeople,\tExtraordinary\tLives\nPENGUIN\tBOOKS'),
 Document(metadata={'source': 'data\\Three Thousand Stitches by Sudha Murthy.pdf'}, page_content='Contents\nPreface\n1.\tThree\tThousand\tStitches\n2.\tHow\tto\tBeat\tthe\tBoys\n3.\tFood\tfor\tThought\n4.\tThree\tHandfuls\tof\tWater\n5.\tCattle\tClass\n6.\tA\tLife\tUnwritten\n7.\tNo\tPlace\tLike\tHome\n8.\tA\tPowerful\tAmbassador\n9.\tRasleela\tand\tthe\tSwimming\tPool\n10.\tA\tDay\tin\tInfosys\tFoundation\n11.\tI\tCan’t,\tWe\tCan\nFollow\tPenguin\nCopyright'),
 Document(metadata={'source': 'data\\Three Thousand Stitches by Sudha Murthy.pdf'}, page_content='PENGUIN\tBOOKS\nTHREE\tTHOUSAN

In [ ]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(  # defined to split text into smaller chunks
        chunk_size=500,
        chunk_overlap=30,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [ ]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 597


In [ ]:
texts_chunk[0]  #Displaying first chunk

Document(metadata={'source': 'data\\Three Thousand Stitches by Sudha Murthy.pdf'}, page_content='SUDHA\tMURTY\n\t\nTHREE\tTHOUSAND\tSTITCHES\nOrdinary\tPeople,\tExtraordinary\tLives\nPENGUIN\tBOOKS')

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings

embedding = download_embeddings()

C:\Users\mahen\AppData\Local\Temp\ipykernel_14048\2457725320.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [ ]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
# vector = embedding.embed_query("Hello world")
# # vector

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get API keys
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")  # ✅ This line was missing

# Set them in environment (optional, if needed by other libraries)
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [ ]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

pc

In [ ]:
import pinecone
from pinecone import Pinecone

# Initialize Pinecone with the new syntax
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = "chatbot"

# List existing indexes
existing_indexes = pc.list_indexes()
print("Existing indexes:", existing_indexes)

# If you need to create an index, use:
# pc.create_index(
#     name=index_name,
#     dimension=1536,  # Adjust based on your embeddings dimension
#     metric='cosine',
#     spec=ServerlessSpec(
#         cloud='aws',
#         region='us-east-1'
#     )
# )

# To connect to an existing index
# index = pc.Index(index_name)

Existing indexes: {'indexes': []}


In [ ]:
from pinecone import Pinecone, ServerlessSpec

index_name = "chatbot"

# Check if index exists by listing all indexes
existing_indexes = pc.list_indexes().names()

if index_name not in existing_indexes:
    print(f"Creating index: {index_name}")
    pc.create_index(
        name=index_name,
        dimension=384,  # Dimension of the embeddings
        metric="cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"Index '{index_name}' created successfully!")
else:
    print(f"Index '{index_name}' already exists.")

# Connect to the index
index = pc.Index(index_name)
print(f"Connected to index: {index_name}")

Index 'chatbot' already exists.
Connected to index: chatbot
